In [27]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from datetime import datetime, timedelta

model_name = 'ecmwfsipn'
model_type = 'reforecast'
filepath = '/home/disk/sipn/nicway/data/model/{model_name}/{model_type}/sipn_nc_agg'.format(model_name = model_name,
                                                                                            model_type = model_type)
print(filepath)
filenames = xr.open_mfdataset(filepath+'/*.nc',concat_dim='init_time')
print(filenames)

/home/disk/sipn/nicway/data/model/ecmwfsipn/reforecast/sipn_nc_agg
<xarray.Dataset>
Dimensions:       (ensemble: 25, fore_time: 215, init_time: 288, nregions: 15)
Coordinates:
  * ensemble      (ensemble) int32 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24
  * fore_time     (fore_time) timedelta64[ns] 1 days 2 days ... 215 days
  * nregions      (nregions) int64 99 2 3 4 5 6 7 8 9 10 11 12 13 14 15
    region_names  (nregions) object 'panArctic' ... 'Central Arctic'
  * init_time     (init_time) datetime64[ns] 1993-01-01 ... 2016-12-01
Data variables:
    Extent        (init_time, ensemble, fore_time, nregions) float64 dask.array<shape=(288, 25, 215, 15), chunksize=(1, 25, 215, 15)>


In [90]:
init_select_full = np.arange(10,15)
region_names = filenames.region_names
init_times = filenames.init_time
forecast_times = filenames.fore_time
extent = filenames.Extent
region_select = 0
no_ens = 25
no_forecast_periods = 6
no_day_change = 5
delta_lead_days = np.arange(5,(no_day_change*no_forecast_periods)+1,no_day_change)
delta_last_days = np.arange(1,no_day_change*no_forecast_periods,no_day_change)
#Create dataframe outside loop
tha_big_D = pd.DataFrame(columns=["I (init date)","V (valid date)","d_SIC (V - I)"])
for itime in init_select_full:
    init_select = itime
    d_SIC_lead_time = pd.DataFrame({"I (init date)":np.tile(init_times[init_select],no_forecast_periods*no_ens),
                                "V (valid date)":"","d_SIC (V - I)":""})
#convert to days
    for iens in np.arange(0,no_ens):
        ens_select = iens
        save_ind = iens*no_forecast_periods+ np.arange(0,no_forecast_periods)
        I_test = extent[init_select,ens_select,:,region_select]
        delta_extent = I_test[delta_lead_days].values - I_test[delta_last_days].values
        d_SIC_lead_time['d_SIC (V - I)'].iloc[save_ind] = delta_extent
        for i in np.arange(0,len(delta_lead_days)):
            ival = init_times[init_select] + pd.Timedelta(delta_lead_days[i],unit='D')
            write_ind = save_ind[i]
            d_SIC_lead_time['V (valid date)'].iloc[write_ind] = pd.to_datetime(ival.values).date()
    #print(ival)
 #   valid_date = np.append(valid_date,ival)
    #print(d_SIC_lead_time)
    tha_big_D = tha_big_D.append(d_SIC_lead_time)
#array_output = 
#fart = tha_big_D.append(d_SIC_lead_time)
#valid_df = pd.DataFrame(np.tile(init_times[init_select],6)) + valid_date
#testfart = pd.date_range(start=init_df.iloc[0],end=end_date,freq='D')

In [94]:
print(tha_big_D.iloc[500:725,:])
print(6*5*25)

    I (init date) V (valid date) d_SIC (V - I)
50     1994-02-01     1994-02-16  -0.000456905
51     1994-02-01     1994-02-21    -0.0241885
52     1994-02-01     1994-02-26   -0.00913343
53     1994-02-01     1994-03-03    -0.0718228
54     1994-02-01     1994-02-06    -0.0529958
55     1994-02-01     1994-02-11      0.155451
56     1994-02-01     1994-02-16     -0.112358
57     1994-02-01     1994-02-21     0.0745233
58     1994-02-01     1994-02-26      0.104823
59     1994-02-01     1994-03-03     -0.145172
60     1994-02-01     1994-02-06     0.0221194
61     1994-02-01     1994-02-11     0.0219906
62     1994-02-01     1994-02-16      0.153995
63     1994-02-01     1994-02-21     0.0414959
64     1994-02-01     1994-02-26     0.0178615
65     1994-02-01     1994-03-03     0.0521153
66     1994-02-01     1994-02-06    -0.0203258
67     1994-02-01     1994-02-11      0.265522
68     1994-02-01     1994-02-16     -0.106992
69     1994-02-01     1994-02-21        0.1308
70     1994-0

In [62]:
#valid_test = init_times[init_select] + pd.Timedelta(1,unit='D')
#print(init_times[init_select],valid_test)
print(iens)

24


In [70]:
print(iens*no_forecast_periods+ np.arange(0,no_forecast_periods))
print(write_ind)

[0 1 2 3 4 5]
4


In [95]:
print(extent.shape)

(288, 25, 215, 15)


In [180]:
flat_test = extent[:,0,0:30,0].stack(desired=['init_time','fore_time'])

print(flat_test)
print(flat_test.values)

<xarray.DataArray 'Extent' (desired: 8640)>
dask.array<shape=(8640,), dtype=float64, chunksize=(30,)>
Coordinates:
    ensemble      int32 0
    nregions      int64 99
    region_names  object 'panArctic'
  * desired       (desired) MultiIndex
  - init_time     (desired) datetime64[ns] 1993-01-01 1993-01-01 ... 1993-01-01
  - fore_time     (desired) timedelta64[ns] 1 days 2 days ... 29 days 30 days
[13.18428379 13.22850416 13.26103552 ... 12.41651797 12.45395472
 12.49685983]


In [182]:
fff = flat_test.init_time
print(fff.values)
foreout = flat_test.fore_time
print(foreout)

['1993-01-01T00:00:00.000000000' '1993-01-01T00:00:00.000000000'
 '1993-01-01T00:00:00.000000000' ... '2016-12-01T00:00:00.000000000'
 '2016-12-01T00:00:00.000000000' '2016-12-01T00:00:00.000000000']
<xarray.DataArray 'fore_time' (desired: 8640)>
array([  86400000000000,  172800000000000,  259200000000000, ...,
       2419200000000000, 2505600000000000, 2592000000000000],
      dtype='timedelta64[ns]')
Coordinates:
    ensemble      int32 0
    nregions      int64 99
    region_names  object 'panArctic'
  * desired       (desired) MultiIndex
  - init_time     (desired) datetime64[ns] 1993-01-01 1993-01-01 ... 1993-01-01
  - fore_time     (desired) timedelta64[ns] 1 days 2 days ... 29 days 30 days


In [196]:
in_days = np.arange(1,30,5)*60*60*24*10**9 #in [ns]
out_days = np.arange(5,31,5)*60*60*24*10**9 #in [ns]
#print(in_days,out_days)
tds = fff + pd.Timedelta(delta_lead_days[0],unit='D')
print(tds)
fart = tds.values
print(fart)

<xarray.DataArray 'init_time' (desired: 8640)>
array(['1993-01-06T00:00:00.000000000', '1993-01-06T00:00:00.000000000',
       '1993-01-06T00:00:00.000000000', ..., '2016-12-06T00:00:00.000000000',
       '2016-12-06T00:00:00.000000000', '2016-12-06T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    ensemble      int32 0
    nregions      int64 99
    region_names  object 'panArctic'
  * desired       (desired) MultiIndex
  - init_time     (desired) datetime64[ns] 1993-01-01 1993-01-01 ... 1993-01-01
  - fore_time     (desired) timedelta64[ns] 1 days 2 days ... 29 days 30 days
['1993-01-06T00:00:00.000000000' '1993-01-06T00:00:00.000000000'
 '1993-01-06T00:00:00.000000000' ... '2016-12-06T00:00:00.000000000'
 '2016-12-06T00:00:00.000000000' '2016-12-06T00:00:00.000000000']


In [179]:
print(fff[0].values)
fucking_god_dammit = pd.Timedelta(in_days[0])
print(fucking_god_dammit)
ugh = fff.isin(fucking_god_dammit.nanoseconds)
#print(pd.datetime(in_days[0]).astype('datetime64[ns]'))
print(in_days[0])
print(ugh)

86400000000000 nanoseconds
1 days 00:00:00
86400000000000
<xarray.DataArray 'fore_time' (desired: 750)>
array([False, False, False, ..., False, False, False])
Coordinates:
    nregions      int64 99
    region_names  object 'panArctic'
    init_time     datetime64[ns] 1993-01-01
  * desired       (desired) MultiIndex
  - ensemble      (desired) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
  - fore_time     (desired) timedelta64[ns] 1 days 2 days ... 29 days 30 days
